In [4]:
#!pip install cowin --user

In [38]:
#!pip install fuzzywuzzy

In [5]:
#!pip install python-telegram-bot

In [1]:
from cowin_api import CoWinAPI
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy.fuzz import partial_token_sort_ratio
import telegram
from telegram.ext import Updater
import logging
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
cowin = CoWinAPI()

In [3]:
states = cowin.get_states()
#print(states)

In [4]:
state_dict={}
for x in states['states']:
    state_dict[x['state_name']] = x['state_id']
#state_dict
#Ktaka = state_id =16

In [8]:
state_id = 16
districts = cowin.get_districts(state_id)
print(districts)

{'districts': [{'district_id': 270, 'district_name': 'Bagalkot'}, {'district_id': 276, 'district_name': 'Bangalore Rural'}, {'district_id': 265, 'district_name': 'Bangalore Urban'}, {'district_id': 294, 'district_name': 'BBMP'}, {'district_id': 264, 'district_name': 'Belgaum'}, {'district_id': 274, 'district_name': 'Bellary'}, {'district_id': 272, 'district_name': 'Bidar'}, {'district_id': 271, 'district_name': 'Chamarajanagar'}, {'district_id': 273, 'district_name': 'Chikamagalur'}, {'district_id': 291, 'district_name': 'Chikkaballapur'}, {'district_id': 268, 'district_name': 'Chitradurga'}, {'district_id': 269, 'district_name': 'Dakshina Kannada'}, {'district_id': 275, 'district_name': 'Davanagere'}, {'district_id': 278, 'district_name': 'Dharwad'}, {'district_id': 280, 'district_name': 'Gadag'}, {'district_id': 267, 'district_name': 'Gulbarga'}, {'district_id': 289, 'district_name': 'Hassan'}, {'district_id': 279, 'district_name': 'Haveri'}, {'district_id': 283, 'district_name': 'Ko

In [9]:
dist_dict={}
for y in districts['districts']:
    dist_dict[y['district_name']] = y['district_id']
#dist_dict
# BBMP = 294, 'Bangalore Rural': 276, 'Bangalore Urban': 265 

In [10]:
district_id = 294
date = '10-05-2021'
available_centers = cowin.get_availability_by_district(district_id,date)
#print(available_centers)

In [75]:
#available_centers

In [11]:
def get_available_centers_only(all_centers):
    avail_centers=[]
    for center in all_centers['centers']:
        for x in center['sessions']:
            avail = 0
            if x['available_capacity']!=0:
                avail=1
                break
        if avail == 1:
            avail_centers.append(center)
    return avail_centers
            

In [12]:
available_centers_filter = get_available_centers_only(available_centers)

In [14]:
df_all = pd.DataFrame(available_centers['centers']).set_index("center_id")
df_all.head(1)

,name,address,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions,vaccine_fees
center_id,,,,,,,,,,,,,
418418,Nethaji Circle UPHC,Near BBMP 36 Ward Office Netaji Circle,Karnataka,BBMP,West,560054,13,77,10:00:00,17:00:00,Free,[{'session_id': 'a8a54a32-cd56-4646-86e0-d3998...,NaN


In [15]:
df_filter = pd.DataFrame(available_centers_filter).set_index("center_id")
df_filter.head(1)

,name,address,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions
center_id,,,,,,,,,,,,
418351,Hosahalli UPHC,"4th Cross,M C Layout",Karnataka,BBMP,West,560040,12,77,22:30:00,05:30:00,Free,[{'session_id': '3d9bc175-7a27-4d25-8856-8f412...


In [18]:
#df_filter.iloc[-1,-1]

In [19]:
for x in process.extract("malleshwaram", df_all['address'], limit=2,scorer =partial_token_sort_ratio):
    print(df_all.loc[x[2],'name'])

K C Gen Hospital
Laggere UPHC


### Create bot

In [20]:
with open("cowin_api_key.txt") as f:
    token= f.read()


In [21]:


bot = telegram.Bot(token=token)

In [22]:
#Check connection
print(bot.get_me())

{'id': 1897412321, 'first_name': 'COWIN_VACC_NOTIFY', 'is_bot': True, 'username': 'COWIN_VACC_NOTIFY_bot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': False}


In [23]:
updater = Updater(token=token, use_context=True)

In [24]:
dispatcher = updater.dispatcher

In [25]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

In [26]:
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

In [27]:

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [28]:
def echo(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text)

In [29]:

echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
dispatcher.add_handler(echo_handler)

In [30]:
"""def caps(update, context):
    text_caps = ' '.join(context.args).upper()
    context.bot.send_message(chat_id=update.effective_chat.id, text=text_caps)"""

"def caps(update, context):\n    text_caps = ' '.join(context.args).upper()\n    context.bot.send_message(chat_id=update.effective_chat.id, text=text_caps)"

In [31]:
def vacc_near_me(update, context):
    #text_caps = ' '.join(context.args).upper()
    for x in process.extract(context.args, df_all['address'], limit=2,scorer =partial_token_sort_ratio):
        text = df_all.loc[x[2],'name']
    context.bot.send_message(chat_id=update.effective_chat.id, text=text)

vacc_near_me = CommandHandler('vacc_near_me', vacc_near_me)
dispatcher.add_handler(vacc_near_me)

In [114]:
process.extract("hulimavu", df_all['address'], limit=2,scorer =partial_token_sort_ratio)


[('Kodigehalli Main Road', 62, 407005),
 ('VANIVILLAS HOSPITAL ROAD, SULTAN ROAD, KALASIPALYA', 50, 634261)]

In [32]:
#Runs the bot
updater.start_polling()

2021-05-08 14:44:18,827 - apscheduler.scheduler - INFO - Scheduler started


2021-05-08 14:45:05,495 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\telegram\ext\dispatcher.py", line 447, in process_update
    handler.handle_update(update, self, check, context)
  File "C:\ProgramData\Anaconda3\lib\site-packages\telegram\ext\handler.py", line 160, in handle_update
    return self.callback(update, context)
  File "<ipython-input-31-8961cf8a3a31>", line 3, in vacc_near_me
    for x in process.extract(context.args, df_all['address'], limit=2,scorer =partial_token_sort_ratio):
  File "C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\process.py", line 168, in extract
    return heapq.nlargest(limit, sl, key=lambda i: i[1]) if limit is not None else \
  File "C:\ProgramData\Anaconda3\lib\heapq.py", line 563, in nlargest
    result = [(key(elem), i, elem) for i, elem in zip(range(0, -n, -1), it)]
  File "C:\ProgramData\Anaconda3\lib\heapq.